In [1]:
try:
  import sys
  sys.path.append('../')
  from modules.search_prop import *
except ImportError:
  url = 'https://raw.githubusercontent.com/jhjeong314/Proofmood/main/modules'
  import httpimport
  with httpimport.remote_repo(url):
    from search_prop import *

This script `search_ann.ipynb` handles all 14 propositional inference rules, including:

- LEM
- Repeat
- 6 introduction rules
- 6 elimination rules

As implied by the name `search_ann.ipynb,` each cell in this Jupyter notebook serves the following purposes:

- When supplied with a proof text, it accurately annotates any lines that lack proper annotations within the conclusion formulas, assuming the requisite formulas or subproofs already exist.
- Hypotheses, comments, and blank lines remain unaltered.
- This process does not involve the creation or deletion of formulas or subproofs.


In [2]:
# 'LEM' and 'repeat' rule example.

prf_str = '''
1. B or A  .hyp
2. A imp B .hyp
proves
3. (C imp D) or not (C imp D)
4. B or A
'''
proof = ProofNodeS(parse_fitch(prf_str))
assert isinstance(proof, ProofNodeS)

proof.show_fitch_text()
print()
# Look for the last formula to validate. Among all invalidated formulas,
#   we will first try to validate the last one.
# We want the position of the node and also the principal connective if
#   the formula is non-prime.
print(ret_val := proof.fmla_to_validate())

│1. B or A	 .hyp
│2. A imp B	 .hyp
├─
│3. (C imp D) or not (C imp D)	x 
│4. B or A	x 

(('4', [0, 3]), <Connective.OR: 'or'>)


In [3]:
# Delve into the return value of fmla_to_validate().
((conc_ln, conc_index), p_conn) = ret_val
print(type(conc_index))
print(conc_index)
print(type(conc_ln))
print(conc_ln)
print(type(p_conn))
print(p_conn)


<class 'list'>
[0, 3]
<class 'str'>
4
<enum 'Connective'>
Connective.OR


In [4]:
# Try 'LEM' rule first.
ans = proof.try_LEM(ret_val, verbosity=2)


'LEM' rule for line 4 failed.
│1. B or A	 .hyp
│2. A imp B	 .hyp
├─
│3. (C imp D) or not (C imp D)	x 
│4. B or A	x 



In [5]:
# Next try the 'repeat' rule.
ans = proof.try_repeat(ret_val, verbosity=2)

'repeat' rule successfully applied to line 4.
│1. B or A	 .hyp
│2. A imp B	 .hyp
├─
│3. (C imp D) or not (C imp D)	x 
│4. B or A	✓ repeat 1



In [6]:
print(ret_val := proof.fmla_to_validate())

(('3', [0, 2]), <Connective.OR: 'or'>)


In [7]:
# Try 'LEM' rule first, again.
ans = proof.try_LEM(ret_val, verbosity=1)

'LEM' rule successfully applied to line 3.


In [8]:
# Use proof.search_proof() to automate above.

prf_str = '''
1. B and A  .hyp
2. A imp B .hyp
proves
3. (C imp D) or not (C imp D)
4. B and A
'''
proof = ProofNodeS(parse_fitch(prf_str))
assert isinstance(proof, ProofNodeS)
proof.search_proof(1)
proof.show_fitch_text()

'repeat' rule successfully applied to line 4.
'LEM' rule successfully applied to line 3.

All formulas have been validated.

│1. B and A	 .hyp
│2. A imp B	 .hyp
├─
│3. (C imp D) or not (C imp D)	✓ LEM 
│4. B and A	✓ repeat 1


Within the context of premises, a subproof with the hypothesis $\alpha$ and the last conclusion $\beta$ can be thought of as equivalent to the formula $\alpha \to \beta$.

This equivalence is especially convenient in the following 3 rules:
* or elim
* imp elim
* iff intro

From this point forward, for instance, the following validation is a legitimate application of the 'or elim' rule:

>  {A imp C, B proves C, A or B} \proves C

We updated the code to reflect this change accordingly. (in `Ann.parse()`)

In [9]:
# bot intro (line 4)

# The annotation of line 4 is not blank. 
# Wrong annotation occupies there.
# We fix the wrong annotation of line 4

prf_str = '''
1. .hyp
proves
  2. not A .hyp
  proves
    3. A .hyp
    proves
    4. bot .imp elim 1,2
    5. B .bot elim 4
  6. A imp B .imp intro 3-5
7. not A imp (A imp B) .imp intro 2-6
'''
proof = ProofNodeS(parse_fitch(prf_str))
assert isinstance(proof, ProofNodeS)
proof.search_proof(verbosity=1)
proof.show_fitch_text()

'repeat' rule for line 4 failed.
'bot intro' rule successfully applied to line 4.

All formulas have been validated.

│1. 	 .hyp
├─
││2. not A	 .hyp
│├─
│││3. A	 .hyp
││├─
│││4. bot	✓ bot intro 2,3
│││5. B	✓ bot elim 4
││6. A imp B	✓ imp intro 3-5
│7. not A imp A imp B	✓ imp intro 2-6


In [10]:
# not intro (line 7)

prf_str = '''
1. .hyp
proves
  # comment line takes up a line
  3. not(A or not A) .hyp
  proves
    4. A .hyp
    proves
    5. A or not A .or intro 4
    6. bot .bot intro 5,3
  7. not A .bot elim 6
  8. A or not A .or intro 7
  9. bot .bot intro 8,3
10. A or not A .not elim 2-9
'''
proof = ProofNodeS(parse_fitch(prf_str))
assert isinstance(proof, ProofNodeS)
proof.search_proof(1)
proof.show_fitch_text()

'repeat' rule for line 7 failed.
'not intro' rule successfully applied to line 7.

All formulas have been validated.

│1. 	 .hyp
├─
││2. # comment line takes up a line
││3. not (A or not A)	 .hyp
│├─
│││4. A	 .hyp
││├─
│││5. A or not A	✓ or intro 4
│││6. bot	✓ bot intro 5,3
││7. not A	✓ not intro 4-6
││8. A or not A	✓ or intro 7
││9. bot	✓ bot intro 8,3
│10. A or not A	✓ not elim 2-9


In [11]:
# and intro (line 4)
# or intro (line 5)
# imp intro (line 6)

prf_str = '''
A .hyp
B .hyp
proves
  C .hyp
  proves
  4. A and B
  5. (A and B) or C
6. C imp (A and B) or C
'''
proof = ProofNodeS(parse_fitch(prf_str))
assert isinstance(proof, ProofNodeS)
proof.search_proof(1)
proof.show_fitch_text()

'repeat' rule for line 6 failed.
'imp intro' rule successfully applied to line 6.
'LEM' rule for line 5 failed.
'repeat' rule for line 5 failed.
'or intro' rule successfully applied to line 5.
'repeat' rule for line 4 failed.
'and intro' rule successfully applied to line 4.

All formulas have been validated.

│1. A	 .hyp
│2. B	 .hyp
├─
││3. C	 .hyp
│├─
││4. A and B	✓ and intro 1,2
││5. (A and B) or C	✓ or intro 4
│6. C imp (A and B) or C	✓ imp intro 3-5


In [12]:
# iff intro (line 4)

prf_str = '''
A .hyp
A imp B .hyp
proves
  B .hyp
  proves
  A
A iff B
'''
proof = ProofNodeS(parse_fitch(prf_str))
assert isinstance(proof, ProofNodeS)
proof.search_proof(1)
proof.show_fitch_text()

'repeat' rule for line 5 failed.
'iff intro' rule successfully applied to line 5.
'repeat' rule successfully applied to line 4.

All formulas have been validated.

│1. A	 .hyp
│2. A imp B	 .hyp
├─
││3. B	 .hyp
│├─
││4. A	✓ repeat 1
│5. A iff B	✓ iff intro 2,3-4


In [13]:
# bot elim (line 4)

prf_str = '''
A .hyp
proves
  not A .hyp
  proves
  bot
  B
not A imp B
'''
proof = ProofNodeS(parse_fitch(prf_str))
assert isinstance(proof, ProofNodeS)
proof.search_proof(1)
proof.show_fitch_text()

'repeat' rule for line 5 failed.
'imp intro' rule successfully applied to line 5.
'repeat' rule for line 4 failed.
'bot elim' rule successfully applied to line 4.
'repeat' rule for line 3 failed.
'bot intro' rule successfully applied to line 3.

All formulas have been validated.

│1. A	 .hyp
├─
││2. not A	 .hyp
│├─
││3. bot	✓ bot intro 1,2
││4. B	✓ bot elim 3
│5. not A imp B	✓ imp intro 2-4


In [14]:
# not elim (line 7)
# imp elim (line 5)

prf_str = '''
1. .hyp
proves
  2. not B imp not A .hyp
  proves
    3. A .hyp
    proves
      4. not B .hyp
      proves
      5. not A 
      6. bot 
    7. B
  8. A imp B 
9. (not B imp not A) imp (A imp B) 
'''
proof = ProofNodeS(parse_fitch(prf_str))
assert isinstance(proof, ProofNodeS)
proof.search_proof(1)
proof.show_fitch_text()

'repeat' rule for line 9 failed.
'imp intro' rule successfully applied to line 9.
'repeat' rule for line 8 failed.
'imp intro' rule successfully applied to line 8.
'repeat' rule for line 7 failed.
'bot elim' rule for line 7 failed.
'not elim' rule successfully applied to line 7.
'repeat' rule for line 6 failed.
'bot intro' rule successfully applied to line 6.
'repeat' rule for line 5 failed.
'not intro' rule for line 5 failed.
'bot elim' rule for line 5 failed.
'not elim' rule for line 5 failed.
'imp elim' rule successfully applied to line 5.

All formulas have been validated.

│1. 	 .hyp
├─
││2. not B imp not A	 .hyp
│├─
│││3. A	 .hyp
││├─
││││4. not B	 .hyp
│││├─
││││5. not A	✓ imp elim 2,4
││││6. bot	✓ bot intro 3,5
│││7. B	✓ not elim 4-6
││8. A imp B	✓ imp intro 3-7
│9. (not B imp not A) imp A imp B	✓ imp intro 2-8


In [15]:
# and elim (line 2)

prf_str = '''
1. A and B .hyp
proves
2. B
'''
proof = ProofNodeS(parse_fitch(prf_str))
assert isinstance(proof, ProofNodeS)
proof.search_proof(1)
proof.show_fitch_text()
  

'repeat' rule for line 2 failed.
'bot elim' rule for line 2 failed.
'not elim' rule for line 2 failed.
'imp elim' rule for line 2 failed.
'iff elim' rule for line 2 failed.
'and elim' rule successfully applied to line 2.

All formulas have been validated.

│1. A and B	 .hyp
├─
│2. B	✓ and elim 1


In [16]:
# or elim (line 8)

prf_str = '''
1. A imp B .hyp
proves
  2. A or C .hyp
  proves
    3. A .hyp
    proves
    4. B
    5. B or C
    6. C .hyp
    proves
    7. B or C
  8. B or C
9. A or C imp B or C
'''
proof = ProofNodeS(parse_fitch(prf_str))
assert isinstance(proof, ProofNodeS)
proof.search_proof(1)
proof.show_fitch_text()

'repeat' rule for line 9 failed.
'imp intro' rule successfully applied to line 9.
'LEM' rule for line 8 failed.
'repeat' rule for line 8 failed.
'or intro' rule for line 8 failed.
'bot elim' rule for line 8 failed.
'not elim' rule for line 8 failed.
'imp elim' rule for line 8 failed.
'iff elim' rule for line 8 failed.
'and elim' rule for line 8 failed.
'or elim' rule successfully applied to line 8.
'LEM' rule for line 7 failed.
'repeat' rule for line 7 failed.
'or intro' rule successfully applied to line 7.
'LEM' rule for line 5 failed.
'repeat' rule for line 5 failed.
'or intro' rule successfully applied to line 5.
'repeat' rule for line 4 failed.
'bot elim' rule for line 4 failed.
'not elim' rule for line 4 failed.
'imp elim' rule successfully applied to line 4.

All formulas have been validated.

│1. A imp B	 .hyp
├─
││2. A or C	 .hyp
│├─
│││3. A	 .hyp
││├─
│││4. B	✓ imp elim 1,3
│││5. B or C	✓ or intro 4
│││6. C	 .hyp
││├─
│││7. B or C	✓ or intro 6
││8. B or C	✓ or elim 2,3-5,6-7
│

In [17]:
# iff elim 

prf_str = '''
1. B .hyp
2. A iff B .hyp
proves
3. A
'''
proof = ProofNodeS(parse_fitch(prf_str))
assert isinstance(proof, ProofNodeS)
proof.search_proof(1)
proof.show_fitch_text()

'repeat' rule for line 3 failed.
'bot elim' rule for line 3 failed.
'not elim' rule for line 3 failed.
'imp elim' rule for line 3 failed.
'iff elim' rule successfully applied to line 3.

All formulas have been validated.

│1. B	 .hyp
│2. A iff B	 .hyp
├─
│3. A	✓ iff elim 1,2
